In [60]:
import pandas as pd
import numpy as np
from scipy.ndimage import gaussian_filter
from scipy.signal import butter, lfilter, cheby1, lfilter_zi, savgol_filter
import matplotlib.pyplot as plt
from ipywidgets import widgets, interactive, Layout, VBox, HBox
from IPython.display import display
from pykalman import KalmanFilter
import pywt

# Load data from an Excel file
def load_data_from_excel(file_path, sheet_name):
    data = pd.read_excel(file_path, sheet_name=sheet_name)
    return data

# Apply a Gaussian filter to the data
def apply_gaussian_filter(data, sigma):
    return gaussian_filter(data, sigma)

# Apply a Butterworth filter to the data
def apply_butterworth_filter(data, N, Wn, btype='low'):
    b, a = butter(N, Wn, btype=btype, analog=False, output='ba')
    zi = lfilter_zi(b, a)
    filtered_data, _ = lfilter(b, a, data, zi=zi*data[0])
    return filtered_data

# Apply a Chebyshev filter to the data
def apply_chebyshev_filter(data, order, rp, Wn, btype='low'):
    b, a = cheby1(order, rp, Wn, btype=btype, analog=False)
    zi = lfilter_zi(b, a)
    filtered_data, _ = lfilter(b, a, data, zi=zi*data[0])
    return filtered_data

# Apply a Savitzky-Golay filter to the data
def apply_savitzky_golay(data, window_length, polyorder):
    return savgol_filter(data, window_length, polyorder)

# Apply Kalman filter to the data
def apply_kalman_filter(data, process_noise, measurement_noise):
    initial_state_mean = [data[0], 0]
    transition_matrix = [[1, 1], [0, 1]]
    observation_matrix = [[1, 0]]
    initial_state_covariance = np.eye(2)  # Identity matrix
    transition_covariance = np.eye(2) * process_noise
    observation_covariance = np.eye(1) * measurement_noise

    kf = KalmanFilter(initial_state_mean=initial_state_mean,
                      initial_state_covariance=initial_state_covariance,
                      transition_matrices=transition_matrix,
                      observation_matrices=observation_matrix,
                      observation_covariance=observation_covariance,
                      transition_covariance=transition_covariance)

    filtered_state_means, _ = kf.filter(data)
    return filtered_state_means[:, 0]

# Apply Wavelet transform to the data
def apply_wavelet_transform(data, wavelet_name, wavelet_level):
    coeffs = pywt.wavedec(data, wavelet_name, level=wavelet_level)
    return pywt.waverec(coeffs, wavelet_name)

# Apply Exponential Moving Average (EMA) to the data
def apply_ema_filter(data, span):
    return data.ewm(span=span).mean()

# Create an interactive plot with zoom buttons
def interactive_plot(x, y,
                     show_gaussian, sigma,
                     show_butterworth, butterworth_N, butterworth_Wn, butterworth_btype, 
                     show_chebyshev, order, chebyshev_rp, chebyshev_Wn, chebyshev_btype, 
                     show_savitzky, window_length, polyorder,
                     show_kalman, process_noise, measurement_noise,
                     show_wavelet, wavelet_name, wavelet_level,
                     show_ema, ema_span, zoom_factor, line_width):
    
    min_len = min(len(x), len(y))
    x = x[:min_len]
    y = y[:min_len]
    
    # Apply Gaussian filter
    gaussian_filtered = apply_gaussian_filter(y, sigma) if show_gaussian else None

    # Apply Butterworth filter
    butterworth_filtered = apply_butterworth_filter(y, butterworth_N, butterworth_Wn, butterworth_btype) if show_butterworth else None

    # Apply Chebyshev filter
    chebyshev_filtered = apply_chebyshev_filter(y, order, chebyshev_rp, chebyshev_Wn, chebyshev_btype) if show_chebyshev else None

    # Apply Savitzky-Golay filter
    savitzky_filtered = apply_savitzky_golay(y, window_length, polyorder) if show_savitzky else None

    # Apply Kalman filter
    kalman_filtered = apply_kalman_filter(y, process_noise, measurement_noise) if show_kalman else None
    if kalman_filtered is not None:
        kalman_filtered = kalman_filtered[:len(y)]  # Trim to match original data length

    # Apply Wavelet transform
    wavelet_filtered = apply_wavelet_transform(y, wavelet_name, wavelet_level) if show_wavelet else None
    if wavelet_filtered is not None:
        wavelet_filtered = wavelet_filtered[:len(y)]  # Trim to match original data length

    # Apply Exponential Moving Average (EMA)
    ema_filtered = apply_ema_filter(pd.Series(y), ema_span) if show_ema else None
    if ema_filtered is not None:
        ema_filtered = ema_filtered[:len(y)]  # Trim to match original data length

    fig_width = 15 * zoom_factor
    fig_height = 6 * zoom_factor
    
    plt.figure(figsize=(fig_width, fig_height))
    
    # Set y-axis limits to match the range of the original data
    y_min = y.min()
    y_max = y.max()
    plt.ylim(y_min, y_max)
    
    # Set x-axis limits to match the original data
    plt.xlim(x.min(), x.max())
    
    plt.plot(x, y, label="Original Data", color='black', lw=.25)
    
    if show_gaussian:
        plt.plot(x, gaussian_filtered, label=f"Gaussian", color='r', lw=line_width)
    if show_butterworth:
        plt.plot(x, butterworth_filtered, label=f"Butterworth", color='o', lw=line_width)
    if show_chebyshev:
        plt.plot(x, chebyshev_filtered, label=f"Chebyshev", color='y', lw=line_width)
    if show_savitzky:
        plt.plot(x, savitzky_filtered, label=f"Savitzky Golay", color='g', lw=line_width)
    if show_kalman:
        plt.plot(x, kalman_filtered, label=f"Kalman", color='b', lw=line_width)
    if show_wavelet:
        plt.plot(x, wavelet_filtered, label=f"Wavelet", color='indigo', lw=line_width)
    if show_ema:
        plt.plot(x, ema_filtered, label=f"EMA", color='violet', lw=line_width)
    
    plt.legend()
    plt.xlabel("X-axis")
    plt.ylabel("Y-axis")
    plt.title("Filter Demo with Independent Smoothing")
    plt.grid()
    plt.show()

# Load Excel data
file_path = "data.xlsx"  # Replace with your Excel file path
sheet_name = "Sheet1"  # Replace with your sheet name
data = load_data_from_excel(file_path, sheet_name)
x = data['time']  # Replace 'Time' with the name of your time column
y = data['data']  # Replace 'Readings' with the name of your readings column

available_wavelets = pywt.wavelist(kind='discrete')

# Create interactive sliders for all the filter parameters and zoom factor
sigma_slider = widgets.FloatSlider(value=1.0, min=0.1, max=50.0, step=0.1, description='G Sigma')
line_width_slider = widgets.FloatSlider(value=1.0, min=0.5, max=5.0, step=0.1, description='Line Width')
butterworth_N_slider = widgets.IntSlider(value=4, min=1, max=10, step=1, description='B N')
butterworth_Wn_slider = widgets.FloatSlider(value=0.1, min=0.01, max=1.0, step=0.01, description='B Wn')
butterworth_btype_dropdown = widgets.Dropdown(options=['low', 'high', 'band', 'stop'], value='low', description='B Type')
chebychev_order = widgets.IntSlider(value=3, min=1, max=9, step=2, description='C order')
chebyshev_rp_slider = widgets.FloatSlider(value=0.1, min=0.01, max=5.0, step=0.01, description='C rp')
chebyshev_Wn_slider = widgets.FloatSlider(value=0.1, min=0.01, max=1.0, step=0.01, description='C Wn')
chebyshev_btype_dropdown = widgets.Dropdown(options=['low', 'high', 'band', 'stop'], value='low', description='C Type')
savitzky_window_length = widgets.IntSlider(value=5, min=3, max=15, step=2, description='SG Window')
savitzky_polyorder = widgets.IntSlider(value=2, min=0, max=5, step=1, description='SG Polyorder')
kalman_process_noise = widgets.FloatSlider(value=0.01, min=0.001, max=0.1, step=0.001, description='Kalman Process Noise')
kalman_measurement_noise = widgets.FloatSlider(value=0.1, min=0.01, max=1.0, step=0.01, description='Kalman Measurement Noise')
wavelet_name_dropdown = widgets.Dropdown(options=['haar', 'db', 'sym', 'coif', 'bior', 'rbio', 'dmey'], value='haar', description='Wavelet')
wavelet_level_slider = widgets.IntSlider(value=1, min=1, max=5, step=1, description='Wavelet Level')
ema_span_slider = widgets.IntSlider(value=10, min=1, max=50, step=1, description='EMA Span')
zoom_slider = widgets.FloatSlider(value=1.0, min=0.5, max=3.0, step=0.1, description='Zoom')

# Create checkboxes for each filter type
gaussian_checkbox = widgets.Checkbox(value=False, description='Show Gaussian Filter')
butterworth_checkbox = widgets.Checkbox(value=False, description='Show Butterworth Filter')
chebyshev_checkbox = widgets.Checkbox(value=False, description='Show Chebyshev Filter')
savitzky_checkbox = widgets.Checkbox(value=False, description='Show Savitzky-Golay Filter')
kalman_checkbox = widgets.Checkbox(value=False, description='Show Kalman Filter')
wavelet_checkbox = widgets.Checkbox(value=False, description='Show Wavelet Transform')
ema_checkbox = widgets.Checkbox(value=False, description='Show Exponential Moving Average')

# Display the interactive plot with sliders, checkboxes, and order slider
interactive_plot_widget = interactive(
    interactive_plot,
    x=widgets.fixed(x),
    y=widgets.fixed(y),
    sigma=sigma_slider,
    line_width=line_width_slider,
    butterworth_N=butterworth_N_slider,
    butterworth_Wn=butterworth_Wn_slider,
    butterworth_btype=butterworth_btype_dropdown,
    order=chebychev_order,
    chebyshev_rp=chebyshev_rp_slider,
    chebyshev_Wn=chebyshev_Wn_slider,
    chebyshev_btype=chebyshev_btype_dropdown,
    window_length=savitzky_window_length,
    polyorder=savitzky_polyorder,
    show_kalman=kalman_checkbox,
    process_noise=kalman_process_noise,
    measurement_noise=kalman_measurement_noise,
    show_wavelet=wavelet_checkbox,
    wavelet_name=wavelet_name_dropdown,
    wavelet_level=wavelet_level_slider,
    show_ema=ema_checkbox,
    ema_span=ema_span_slider,
    zoom_factor=zoom_slider,
    show_gaussian=gaussian_checkbox,
    show_butterworth=butterworth_checkbox,
    show_chebyshev=chebyshev_checkbox,
    show_savitzky=savitzky_checkbox,
)

# Display the widgets using a VBox
display(VBox([interactive_plot_widget]))
